In [1]:
from kaggle_secrets import UserSecretsClient
from huggingface_hub import login

user_secrets = UserSecretsClient()
hf_token = user_secrets.get_secret("HF_TOKEN")

login(token=hf_token)

In [2]:
import torch
import torch.nn as nn
from transformers import AutoModelForCausalLM, AutoTokenizer
from huggingface_hub import hf_hub_download
from safetensors.torch import load_file

torch.set_grad_enabled(False)

class JumpReLUSAE(nn.Module):
    def __init__(self, d_in, d_sae):
        super().__init__()
        self.w_enc = nn.Parameter(torch.zeros(d_in, d_sae))
        self.w_dec = nn.Parameter(torch.zeros(d_sae, d_in))
        self.threshold = nn.Parameter(torch.zeros(d_sae))
        self.b_enc = nn.Parameter(torch.zeros(d_sae))
        self.b_dec = nn.Parameter(torch.zeros(d_in))
    def encode(self, x):
        pre = x @ self.w_enc + self.b_enc
        return (pre > self.threshold) * torch.relu(pre)
    def decode(self, acts): return acts @ self.w_dec + self.b_dec
    def forward(self, x): return self.decode(self.encode(x))

# Модель
tokenizer = AutoTokenizer.from_pretrained("google/gemma-3-4b-pt")
model = AutoModelForCausalLM.from_pretrained(
    "google/gemma-3-4b-pt", dtype=torch.bfloat16, device_map="cuda:0"
)
model.eval()
print("✓ model loaded")

# SAE
path = hf_hub_download(
    repo_id="google/gemma-scope-2-4b-pt",
    filename="resid_post/layer_22_width_65k_l0_medium/params.safetensors"
)
params = load_file(path)
d_model, d_sae = params["w_enc"].shape
sae = JumpReLUSAE(d_model, d_sae)
sae.load_state_dict(params)
sae = sae.to("cuda", dtype=torch.bfloat16)
sae.eval()
print(f"✓ SAE loaded  d_model={d_model}  d_sae={d_sae}")

config.json:   0%|          | 0.00/815 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

...

Loading weights:   0%|          | 0/883 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

✓ model loaded


resid_post/layer_22_width_65k_l0_medium/(…):   0%|          | 0.00/1.34G [00:00<?, ?B/s]

✓ SAE loaded  d_model=2560  d_sae=65536


In [3]:
debug_cache = {}

def debug_hook(mod, inp, out):
    print(f"type(out)     : {type(out)}")
    print(f"out[0].shape  : {out[0].shape}")
    print(f"out[0].dtype  : {out[0].dtype}")
    print(f"has NaN       : {torch.isnan(out[0]).any()}")
    print(f"min/max       : {out[0].min():.4f} / {out[0].max():.4f}")
    debug_cache["raw"] = out[0].detach().cpu()

# Находим слои
inner = model.model
if hasattr(inner, "language_model"):
    inner = inner.language_model
layers = inner.layers
print(f"Layers type: {type(layers)}, count: {len(layers)}")

# Тестовый прогон
text = "Он педантичный человек, который всё делает по правилам."
tokens = tokenizer.encode(text, return_tensors="pt", add_special_tokens=True).to("cuda")
print(f"Input shape: {tokens.shape}")

handle = layers[22].register_forward_hook(debug_hook)
try:
    model(tokens)
finally:
    handle.remove()

# Проверка SAE вручную
raw = debug_cache["raw"]
print(f"\nraw shape: {raw.shape} → dim={raw.dim()}")
resid = raw.squeeze(0).float() if raw.dim() == 3 else raw.float()
print(f"resid after squeeze: {resid.shape}")

pre_acts = resid @ sae.w_enc.cpu().float() + sae.b_enc.cpu().float()
print(f"pre_acts: shape={pre_acts.shape}, min={pre_acts.min():.3f}, max={pre_acts.max():.3f}, NaN={torch.isnan(pre_acts).any()}")

acts = (pre_acts > sae.threshold.cpu().float()) * torch.relu(pre_acts)
print(f"sae_acts nonzero per token: {(acts > 0).sum(-1).tolist()}")
print(f"top-5 latents by mean: {acts[1:].mean(0).topk(5)}")

Layers type: <class 'torch.nn.modules.container.ModuleList'>, count: 34
Input shape: torch.Size([1, 15])
type(out)     : <class 'torch.Tensor'>
out[0].shape  : torch.Size([15, 2560])
out[0].dtype  : torch.bfloat16
has NaN       : False
min/max       : -6624.0000 / 274432.0000

raw shape: torch.Size([15, 2560]) → dim=2
resid after squeeze: torch.Size([15, 2560])
pre_acts: shape=torch.Size([15, 65536]), min=-60864.688, max=18292.082, NaN=False
sae_acts nonzero per token: [707, 39, 40, 47, 60, 68, 61, 50, 53, 82, 77, 67, 68, 70, 57]
top-5 latents by mean: torch.return_types.topk(
values=tensor([1224.2137,  689.3002,  597.4240,  585.3576,  511.3463]),
indices=tensor([1263, 8066, 1491,  542,   98]))


In [ ]:
import polars as pl

def _get_layers(model):
    inner = model.model
    if hasattr(inner, "language_model"):
        inner = inner.language_model
    return inner.layers

def get_residual_activations(text):
    tokens = tokenizer.encode(text, return_tensors="pt", add_special_tokens=True).to("cuda")
    cache = {}
    def hook(mod, inp, out):
        # out is Tensor (1, seq, d_model) — берём [0] → (seq, d_model)
        cache["acts"] = out[0].detach().float()
    handle = _get_layers(model)[22].register_forward_hook(hook)
    try:
        model(tokens)
    finally:
        handle.remove()
    return cache["acts"]  # (seq, d_model)

def get_sae_acts(text):
    resid = get_residual_activations(text)          # (seq, d_model)
    acts = sae.encode(resid.to(sae.w_enc.dtype))   # (seq, d_sae)
    valid = acts[1:].float()                        # skip BOS
    return {
        "mean": valid.mean(0).cpu(),     # (d_sae,)
        "max":  valid.max(0).values.cpu(),
        "fired_frac": (valid > 0).float().mean(0).cpu(),
        "per_token": acts.cpu(),
        "tokens": tokenizer.convert_ids_to_tokens(
            tokenizer.encode(text, add_special_tokens=True)
        )
    }

print("✓ helpers ready")

✓ helpers ready


In [ ]:
# df = pl.read_csv("data\psych_trait_dataset_v2_clean.csv") for local
df = pl.read_csv("/kaggle/input/datasets/marii8st/psycho-scope-dataset-traits/psych_trait_dataset_v2_clean.csv")
df = df.filter(pl.col("validation_passed") == True)

TOP_K = 20
trait_to_latents = {}

for trait in df["trait"].unique().sort().to_list():
    texts = df.filter(
        (pl.col("trait") == trait) & (pl.col("data_type") == "A_explicit")
    )["text"].to_list()
    
    import torch
    accum = torch.zeros(65536)
    for text in texts:
        out = get_sae_acts(text)
        accum += out["mean"]
    accum /= len(texts)
    
    top_vals, top_idx = accum.topk(TOP_K)
    trait_to_latents[trait] = top_idx.tolist()
    print(f"{trait}: top-5 latents {top_idx[:5].tolist()} | acts {top_vals[:5].round().tolist()}")

Agreeableness: top-5 latents [966, 839, 355, 1263, 273] | acts [691.0, 331.0, 318.0, 307.0, 226.0]
Conscientiousness: top-5 latents [966, 839, 1263, 355, 72] | acts [731.0, 335.0, 302.0, 237.0, 205.0]
Extraversion: top-5 latents [966, 839, 1263, 355, 1323] | acts [385.0, 303.0, 297.0, 242.0, 163.0]
Narcissism: top-5 latents [966, 839, 1263, 50, 355] | acts [528.0, 322.0, 302.0, 240.0, 236.0]
Neuroticism: top-5 latents [966, 13606, 1263, 839, 745] | acts [414.0, 336.0, 298.0, 296.0, 271.0]
Openness: top-5 latents [966, 839, 1263, 355, 1501] | acts [597.0, 365.0, 301.0, 248.0, 212.0]


In [25]:
# Смотрим unembedding — какие слова предсказывает каждый латент
w_u = model.lm_head.weight.float().cpu()   # (vocab, d_model)

# Для Gemma 3 norm веса могут быть внутри language_model
try:
    norm_w = model.model.language_model.norm.weight.float().cpu()
except:
    norm_w = model.model.norm.weight.float().cpu()

w_u_eff = w_u * norm_w  # (vocab, d_model)

for trait, latents in trait_to_latents.items():
    lidx = latents[0]
    dec_vec = sae.w_dec[lidx].float().cpu()  # (d_model,)
    scores = w_u_eff @ dec_vec               # (vocab,)
    top_toks = scores.topk(15).indices
    words = [tokenizer.decode(t) for t in top_toks]
    print(f"\n{trait} (latent {lidx}):")
    print(f"  top words: {words}")


Agreeableness (latent 966):
  top words: ['🌐', '🔄', '🔍', '💼', ' meticulously', 'Notably', '📜', '🏞', '🌍', '🌟', ' Notably', ' unwavering', ' intricately', '🏰', ' esteemed']

Conscientiousness (latent 966):
  top words: ['🌐', '🔄', '🔍', '💼', ' meticulously', 'Notably', '📜', '🏞', '🌍', '🌟', ' Notably', ' unwavering', ' intricately', '🏰', ' esteemed']

Extraversion (latent 966):
  top words: ['🌐', '🔄', '🔍', '💼', ' meticulously', 'Notably', '📜', '🏞', '🌍', '🌟', ' Notably', ' unwavering', ' intricately', '🏰', ' esteemed']

Narcissism (latent 966):
  top words: ['🌐', '🔄', '🔍', '💼', ' meticulously', 'Notably', '📜', '🏞', '🌍', '🌟', ' Notably', ' unwavering', ' intricately', '🏰', ' esteemed']

Neuroticism (latent 966):
  top words: ['🌐', '🔄', '🔍', '💼', ' meticulously', 'Notably', '📜', '🏞', '🌍', '🌟', ' Notably', ' unwavering', ' intricately', '🏰', ' esteemed']

Openness (latent 966):
  top words: ['🌐', '🔄', '🔍', '💼', ' meticulously', 'Notably', '📜', '🏞', '🌍', '🌟', ' Notably', ' unwavering', ' intrica

In [11]:
# Правильный discovery: контраст A_explicit vs среднее по всем другим трейтам
# Ищем латенты СПЕЦИФИЧНЫЕ для данного трейта

import torch
import polars as pl

def discover_contrastive(df, target_trait, top_k=20):
    """
    Finds latents that activate strongly for target_trait A_explicit
    but weakly for all other traits A_explicit.
    """
    # Среднее по target trait
    target_texts = df.filter(
        (pl.col("trait") == target_trait) & 
        (pl.col("data_type") == "A_explicit")
    )["text"].to_list()
    
    target_mean = torch.zeros(65536)
    for text in target_texts:
        target_mean += get_sae_acts(text)["mean"]
    target_mean /= len(target_texts)
    
    # Среднее по всем остальным трейтам (A_explicit)
    other_texts = df.filter(
        (pl.col("trait") != target_trait) & 
        (pl.col("data_type") == "A_explicit")
    )["text"].to_list()
    
    other_mean = torch.zeros(65536)
    for text in other_texts:
        other_mean += get_sae_acts(text)["mean"]
    other_mean /= len(other_texts)
    
    # Контрастный скор: target минус все остальные
    contrast = target_mean - other_mean
    top_vals, top_idx = contrast.topk(top_k)
    return top_idx.tolist(), top_vals.tolist()


# Запускаем для всех трейтов
print("Contrastive discovery (trait-specific latents):\n")
trait_to_latents_v2 = {}

for trait in df["trait"].unique().sort().to_list():
    latents, vals = discover_contrastive(df, trait, top_k=20)
    trait_to_latents_v2[trait] = latents
    
    # Показываем топ-5 с unembedding
    print(f"{trait}:")
    for idx, val in zip(latents[:5], vals[:5]):
        dec_vec = sae.w_dec[idx].float().cpu()
        words = [tokenizer.decode(t) for t in (w_u_eff @ dec_vec).topk(5).indices]
        print(f"  latent={idx:6d}  Δ={val:5.1f}  words={words}")
    print()

Contrastive discovery (trait-specific latents):

Agreeableness:
  latent=   273  Δ=182.0  words=[' compassion', ' kindness', ' empathy', ' friendship', ' friendships']
  latent=   966  Δ=159.5  words=['🌐', '🔄', '🔍', '💼', ' meticulously']
  latent=  3121  Δ= 87.7  words=[' juntos', ' together', ' jointly', '共同', ' collaboratively']
  latent=   355  Δ= 82.9  words=['.', '.\\\\', '.\u200b', '.”', '.\ufeff']
  latent=  2296  Δ= 70.3  words=[' patiently', ' expertly', ' personable', ' tirelessly', ' thoughtful']

Conscientiousness:
  latent=   966  Δ=208.2  words=['🌐', '🔄', '🔍', '💼', ' meticulously']
  latent=    72  Δ=180.0  words=[' meticulously', ' meticulous', ' meticul', ' painstakingly', ' painstaking']
  latent=  4056  Δ= 78.0  words=[' unwavering', ' delectable', ' captivating', ' pivotal', ' exhilarating']
  latent=  9012  Δ= 69.8  words=['理性', ' impersonal', ' scientific', 'scientific', '科学']
  latent=  1394  Δ= 60.2  words=[' ounce', ' nook', ' einzelnen', ' einzelne', ' conceiva

In [12]:
# Measurement с контрастными латентами
records_v2 = []

for row in df.filter(pl.col("validation_passed") == True).iter_rows(named=True):
    trait = row["trait"]
    latents = trait_to_latents_v2.get(trait, [])
    out = get_sae_acts(row["text"])
    
    for rank, lidx in enumerate(latents[:5]):
        records_v2.append({
            "trait":       trait,
            "data_type":   row["data_type"],
            "pole":        row["pole"],
            "topic":       row["topic"],
            "rep":         row.get("rep", 0),
            "latent_idx":  lidx,
            "latent_rank": rank,
            "mean_act":    out["mean"][lidx].item(),
            "max_act":     out["max"][lidx].item(),
            "fired_frac":  out["fired_frac"][lidx].item(),
        })

mdf2 = pl.DataFrame(records_v2)
mdf2.write_parquet("/kaggle/working/activations_v2.parquet")
print(f"✓ {len(mdf2)} rows")

# Hypothesis test
print("\n── HYPOTHESIS TEST v2: B_implicit vs C_baseline ──────────────")
print(f"{'Trait':<20} {'Latent':>8} {'Words':<35} {'B':>7} {'C':>7} {'Δ':>7} {'ratio':>7}")
print("─" * 95)

for trait in sorted(trait_to_latents_v2):
    latents = trait_to_latents_v2[trait]
    
    for rank, lidx in enumerate(latents[:3]):  # топ-3 латента
        b_vals = mdf2.filter(
            (pl.col("trait") == trait) &
            (pl.col("data_type") == "B_implicit") &
            (pl.col("latent_rank") == rank)
        )["mean_act"]
        
        c_vals = mdf2.filter(
            (pl.col("trait") == trait) &
            (pl.col("data_type") == "C_baseline") &
            (pl.col("latent_rank") == rank)
        )["mean_act"]
        
        if len(b_vals) == 0 or len(c_vals) == 0:
            continue
            
        b_mean = b_vals.mean()
        c_mean = c_vals.mean()
        delta  = b_mean - c_mean
        ratio  = b_mean / (c_mean + 1e-9)
        
        # Слова латента
        dec_vec = sae.w_dec[lidx].float().cpu()
        words = [tokenizer.decode(t) for t in (w_u_eff @ dec_vec).topk(3).indices]
        words_str = ",".join(words)[:33]
        
        marker = " ★" if ratio > 5 else ("" if ratio > 2 else " ✗")
        print(f"{trait:<20} {lidx:>8} {words_str:<35} {b_mean:>7.1f} {c_mean:>7.1f} {delta:>7.1f} {ratio:>6.1f}x{marker}")
    print()

✓ 5400 rows

── HYPOTHESIS TEST v2: B_implicit vs C_baseline ──────────────
Trait                  Latent Words                                     B       C       Δ   ratio
───────────────────────────────────────────────────────────────────────────────────────────────
Agreeableness             273  compassion, kindness, empathy        107.3    18.5    88.8    5.8x ★
Agreeableness             966 🌐,🔄,🔍                                  70.4    22.0    48.4    3.2x
Agreeableness            3121  juntos, together, jointly             53.3     8.8    44.5    6.1x ★

Conscientiousness         966 🌐,🔄,🔍                                 439.2     1.9   437.3  234.5x ★
Conscientiousness          72  meticulously, meticulous, meticu      90.0     4.1    86.0   22.1x ★
Conscientiousness        4056  unwavering, delectable, captivat      83.4     0.0    83.4 83353042976.1x ★

Extraversion             7553  networking, Networking,Networkin      30.0     3.1    26.9    9.6x ★
Extraversion           

In [13]:
from scipy import stats
import numpy as np

print("\n── STATISTICAL SIGNIFICANCE ───────────────────────────────────")
print(f"{'Trait':<20} {'Latent':>8} {'B_mean':>8} {'C_mean':>8} {'ratio':>7} {'p-value':>10} {'sig':>5}")
print("─" * 75)

# Выбираем лучший латент для каждого трейта
best_latents = {
    "Neuroticism":        745,
    "Conscientiousness":   72,
    "Extraversion":     23505,
    "Agreeableness":      273,
    "Openness":          3415,
    "Narcissism":        2067,
}

p_values = [1.25e-19, 1.07e-19, 2.25e-20, 7.14e-11, 4.33e-16, 2.96e-07]
n_tests = len(p_values)

results_stats = []

for trait, lidx in best_latents.items():
    # Найдём rank этого латента
    rank = trait_to_latents_v2[trait].index(lidx) if lidx in trait_to_latents_v2[trait] else 0
    
    b_vals = mdf2.filter(
        (pl.col("trait") == trait) &
        (pl.col("data_type") == "B_implicit") &
        (pl.col("latent_rank") == rank)
    )["mean_act"].to_numpy()
    
    c_vals = mdf2.filter(
        (pl.col("trait") == trait) &
        (pl.col("data_type") == "C_baseline") &
        (pl.col("latent_rank") == rank)
    )["mean_act"].to_numpy()
    
    # Mann-Whitney U (не предполагает нормальность)
    stat, p = stats.mannwhitneyu(b_vals, c_vals, alternative="greater")
    
    # Cohen's d
    pooled_std = np.sqrt((b_vals.std()**2 + c_vals.std()**2) / 2)
    cohens_d = (b_vals.mean() - c_vals.mean()) / (pooled_std + 1e-9)
    
    sig = "***" if p < 0.001 else ("**" if p < 0.01 else ("*" if p < 0.05 else "ns"))
    ratio = b_vals.mean() / (c_vals.mean() + 1e-9)
    
    print(f"{trait:<20} {lidx:>8} {b_vals.mean():>8.1f} {c_vals.mean():>8.1f} "
          f"{ratio:>6.1f}x {p:>10.2e} {sig:>5}  d={cohens_d:.2f}")
    
    results_stats.append({
        "trait": trait, "latent_idx": lidx,
        "b_mean": b_vals.mean(), "c_mean": c_vals.mean(),
        "ratio": ratio, "p_value": p, "cohens_d": cohens_d,
        "n_b": len(b_vals), "n_c": len(c_vals),
    })

stats_df = pl.DataFrame(results_stats)
stats_df.write_csv("/kaggle/working/statistical_results.csv")
print(f"\n✓ saved → statistical_results.csv")


── STATISTICAL SIGNIFICANCE ───────────────────────────────────
Trait                  Latent   B_mean   C_mean   ratio    p-value   sig
───────────────────────────────────────────────────────────────────────────
Neuroticism               745    209.5     19.2   10.9x   1.25e-19   ***  d=2.39
Conscientiousness          72     90.0      4.1   22.1x   1.07e-19   ***  d=1.65
Extraversion            23505     47.1      0.6   76.4x   2.25e-20   ***  d=1.53
Agreeableness             273    107.3     18.5    5.8x   7.14e-11   ***  d=1.12
Openness                 3415     46.3      0.1  884.2x   4.33e-16   ***  d=1.19
Narcissism               2067     52.5     19.1    2.8x   2.96e-07   ***  d=0.68

✓ saved → statistical_results.csv


In [24]:
# Показываем ratio по каждому топику для лучшего латента
print("── Per-topic consistency: Neuroticism latent 745 ──\n")

topics = df.filter(pl.col("trait") == "Neuroticism")["topic"].unique().to_list()

topic_results = []
for topic in sorted(topics):
    b = mdf2.filter(
        (pl.col("trait") == "Neuroticism") &
        (pl.col("data_type") == "B_implicit") &
        (pl.col("latent_rank") == 1) &  # rank 1 = latent 745
        (pl.col("topic") == topic)
    )["mean_act"]
    
    c = mdf2.filter(
        (pl.col("trait") == "Neuroticism") &
        (pl.col("data_type") == "C_baseline") &
        (pl.col("latent_rank") == 1) &
        (pl.col("topic") == topic)
    )["mean_act"]
    
    if len(b) and len(c):
        ratio = b.mean() / (c.mean() + 1e-9)
        direction = "✓" if ratio > 2 else "✗"
        topic_results.append((topic, b.mean(), c.mean(), ratio))
        print(f"  {direction} {ratio:5.1f}x  B={b.mean():6.1f}  C={c.mean():5.1f}  {topic[:40]}")

consistent = sum(1 for _, _, _, r in topic_results if r > 2)
print(f"\n  Consistent topics (ratio>2): {consistent}/{len(topic_results)}")

── Per-topic consistency: Neuroticism latent 745 ──

  ✓ 157.1x  B= 318.0  C=  2.0  being late to an important meeting
  ✓   9.5x  B= 188.9  C= 19.8  choosing a new book or TV show to start
  ✓  26.1x  B= 102.9  C=  3.9  cooking dinner after a long day
  ✓ 127729189554.9x  B= 127.7  C=  0.0  dealing with a broken household applianc
  ✓   8.8x  B= 110.3  C= 12.6  deciding what to have for lunch
  ✓   7.7x  B= 148.3  C= 19.3  describing their morning routine
  ✓  11.8x  B= 223.4  C= 19.0  discussing a recent project they complet
  ✓  11.3x  B= 120.1  C= 10.6  explaining how they organise their works
  ✓   8.9x  B=  39.6  C=  4.5  giving directions to a stranger
  ✓  29.4x  B=  72.3  C=  2.5  making a cup of tea or coffee
  ✓  28.2x  B= 320.0  C= 11.4  meeting someone new at a party
  ✓  49.9x  B= 125.7  C=  2.5  planning a weekend trip
  ✓   9.4x  B= 294.0  C= 31.2  reacting to a friend cancelling plans la
  ✓   6.7x  B= 302.2  C= 45.4  reacting to a minor personal failure
  ✓   3.4x  B=

In [23]:
p_values = [1.25e-19, 1.07e-19, 2.25e-20, 7.14e-11, 4.33e-16, 2.96e-07]
n_tests = len(p_values)

print("── Bonferroni correction ──")
for trait, p in zip(best_latents.keys(), p_values):
    p_corrected = min(p * n_tests, 1.0)
    sig = "***" if p_corrected < 0.001 else ("**" if p_corrected < 0.01 else "*")
    print(f"{trait:<20}  p_raw={p:.2e}  p_bonf={p_corrected:.2e}  {sig}")

── Bonferroni correction ──
Neuroticism           p_raw=1.25e-19  p_bonf=7.50e-19  ***
Conscientiousness     p_raw=1.07e-19  p_bonf=6.42e-19  ***
Extraversion          p_raw=2.25e-20  p_bonf=1.35e-19  ***
Agreeableness         p_raw=7.14e-11  p_bonf=4.28e-10  ***
Openness              p_raw=4.33e-16  p_bonf=2.60e-15  ***
Narcissism            p_raw=2.96e-07  p_bonf=1.78e-06  ***


In [15]:
from IPython.display import display, HTML
import numpy as np

def render_token_heatmap(text, latent_idx, trait, data_type, pole, topic):
    """Renders a single text as a token-level activation heatmap."""
    out = get_sae_acts(text)
    tokens = out["tokens"]
    acts = out["per_token"][:, latent_idx].float().numpy()
    max_act = max(acts.max(), 1e-8)
    
    spans = []
    for tok, act in zip(tokens, acts):
        tok_clean = (tok
            .replace("▁", " ")
            .replace("<", "&lt;")
            .replace(">", "&gt;")
            .replace("\n", "↵")
            .replace("'", "&#39;"))
        
        norm = float(act) / max_act
        # Green for fired, gray for zero
        if act > 0:
            r = int(255 * (1 - norm * 0.6))
            g = int(200 + 55 * norm)
            b = int(100 * (1 - norm))
            bg = f"rgb({r},{g},{b})"
            fw = "bold" if norm > 0.5 else "normal"
        else:
            bg = "transparent"
            fw = "normal"
        
        title = f"act={act:.1f}"
        spans.append(
            f'<span style="background:{bg};font-weight:{fw};padding:2px 1px;'
            f'border-radius:2px;margin:1px;display:inline-block;" title="{title}">'
            f'{tok_clean}</span>'
        )
    
    dtype_colors = {
        "A_explicit": "#3d85c8",
        "B_implicit": "#6aa84f", 
        "C_baseline": "#cc4125",
    }
    color = dtype_colors.get(data_type, "#888")
    
    return f"""
<div style="margin:6px 0;padding:8px 12px;border-left:4px solid {color};
            background:#1e1e1e;border-radius:4px;">
  <div style="font-size:11px;color:#aaa;margin-bottom:5px;font-family:monospace;">
    <span style="color:{color};font-weight:bold">{data_type}</span>
    &nbsp;|&nbsp;pole: <b>{pole}</b>
    &nbsp;|&nbsp;topic: <i>{topic[:40]}</i>
    &nbsp;|&nbsp;max_act: <b>{max_act:.1f}</b>
  </div>
  <div style="line-height:2.0;font-family:monospace;font-size:13px;">
    {"".join(spans)}
  </div>
</div>"""


def build_full_report(df, trait_to_latents_v2, best_latents, n_examples=3):
    """
    Builds complete HTML report.
    For each trait: shows top latent, then A/B/C examples side by side.
    """
    sections = []
    
    # Header
    sections.append("""
<div style="font-family:Arial,sans-serif;max-width:1200px;margin:auto;padding:24px;background:#111;color:#ddd;">
<h1 style="color:#7ec8e3;border-bottom:2px solid #333;padding-bottom:12px;">
  🧠 Psychological Trait Activation Report
</h1>
<p style="color:#aaa;">
  Token-level SAE latent activations for implicit personality detection.<br>
  <b style="color:#3d85c8">■ A_explicit</b> &nbsp;
  <b style="color:#6aa84f">■ B_implicit</b> &nbsp;  
  <b style="color:#cc4125">■ C_baseline</b>
  &nbsp;|&nbsp; Green intensity = activation strength
</p>
""")
    
    # Stats table
    sections.append("""
<h2 style="color:#f0a500;margin-top:32px;">Statistical Summary</h2>
<table style="width:100%;border-collapse:collapse;font-size:13px;margin-bottom:24px;">
<tr style="background:#2a2a2a;color:#aaa;">
  <th style="padding:8px;text-align:left;">Trait</th>
  <th style="padding:8px;">Latent</th>
  <th style="padding:8px;">Key Words</th>
  <th style="padding:8px;">B mean</th>
  <th style="padding:8px;">C mean</th>
  <th style="padding:8px;">Ratio</th>
  <th style="padding:8px;">Cohen's d</th>
  <th style="padding:8px;">p-value</th>
</tr>
""")
    
    row_data = [
        ("Neuroticism",       745,   "anguish, sadness, sobbing",      209.5, 19.2,  10.9, 2.39, "1.25e-19"),
        ("Conscientiousness",  72,   "meticulously, painstakingly",     90.0,  4.1,  22.1, 1.65, "1.07e-19"),
        ("Extraversion",    23505,   "laughter, playful, cheerful",     47.1,  0.6,  76.4, 1.53, "2.25e-20"),
        ("Openness",         3415,   "artists, artworks, artistic",     46.3,  0.1, 884.2, 1.19, "4.33e-16"),
        ("Agreeableness",     273,   "compassion, kindness, empathy",  107.3, 18.5,   5.8, 1.12, "7.14e-11"),
        ("Narcissism",       2067,   "arrogant, arrogance",             52.5, 19.1,   2.8, 0.68, "2.96e-07"),
    ]
    
    for trait, lidx, words, b, c, ratio, d, p in row_data:
        d_color = "#6aa84f" if d >= 0.8 else "#f0a500"
        sections.append(f"""
<tr style="border-bottom:1px solid #2a2a2a;">
  <td style="padding:8px;font-weight:bold;">{trait}</td>
  <td style="padding:8px;text-align:center;font-family:monospace;color:#aaa;">{lidx}</td>
  <td style="padding:8px;font-style:italic;color:#ccc;">{words}</td>
  <td style="padding:8px;text-align:center;">{b:.1f}</td>
  <td style="padding:8px;text-align:center;">{c:.1f}</td>
  <td style="padding:8px;text-align:center;font-weight:bold;color:#7ec8e3;">{ratio:.1f}x</td>
  <td style="padding:8px;text-align:center;font-weight:bold;color:{d_color};">{d:.2f}</td>
  <td style="padding:8px;text-align:center;color:#6aa84f;font-family:monospace;">***</td>
</tr>""")
    
    sections.append("</table>")
    
    # Per-trait heatmaps
    sections.append('<h2 style="color:#f0a500;margin-top:32px;">Per-Trait Activation Heatmaps</h2>')
    
    for trait, lidx in best_latents.items():
        sections.append(f"""
<div style="margin:28px 0;padding:16px;border:1px solid #333;border-radius:8px;background:#161616;">
  <h3 style="color:#7ec8e3;margin:0 0 4px 0;">{trait}
    <span style="font-size:13px;color:#888;font-weight:normal;margin-left:12px;">
      latent #{lidx}
    </span>
  </h3>
""")
        
        for dtype in ["A_explicit", "B_implicit", "C_baseline"]:
            subset = (df
                .filter(pl.col("validation_passed") == True)
                .filter(pl.col("trait") == trait)
                .filter(pl.col("data_type") == dtype)
                .head(n_examples))
            
            dtype_label = {
                "A_explicit": "A — Explicit (latent discovery texts)",
                "B_implicit": "B — Implicit (personality without keywords) ← KEY TEST",
                "C_baseline": "C — Baseline (opposite pole, same topic)",
            }[dtype]
            
            sections.append(f'<h4 style="color:#aaa;margin:16px 0 6px 0;font-size:12px;">{dtype_label}</h4>')
            
            for row in subset.iter_rows(named=True):
                sections.append(render_token_heatmap(
                    text=row["text"],
                    latent_idx=lidx,
                    trait=trait,
                    data_type=dtype,
                    pole=row["pole"],
                    topic=row["topic"],
                ))
        
        sections.append("</div>")
    
    sections.append("</div>")  # close main div
    return "\n".join(sections)


# ── Build and save ────────────────────────────────────────────────────────────

print("Building HTML report...")

html_content = build_full_report(
    df=df.filter(pl.col("validation_passed") == True),
    trait_to_latents_v2=trait_to_latents_v2,
    best_latents=best_latents,
    n_examples=3,
)

# Save to file
out_path = "/kaggle/working/activation_report_final.html"
with open(out_path, "w", encoding="utf-8") as f:
    f.write(html_content)

print(f"✓ Saved → {out_path}")

# Preview in notebook (first trait only to avoid overloading the cell output)
preview = build_full_report(
    df=df.filter(
        pl.col("validation_passed") == True
    ).filter(
        pl.col("trait") == "Neuroticism"
    ),
    trait_to_latents_v2={"Neuroticism": trait_to_latents_v2["Neuroticism"]},
    best_latents={"Neuroticism": best_latents["Neuroticism"]},
    n_examples=2,
)
display(HTML(preview))

Building HTML report...
✓ Saved → /kaggle/working/activation_report_final.html


Trait,Latent,Key Words,B mean,C mean,Ratio,Cohen's d,p-value
Neuroticism,745,"anguish, sadness, sobbing",209.5,19.2,10.9x,2.39,***
Conscientiousness,72,"meticulously, painstakingly",90.0,4.1,22.1x,1.65,***
Extraversion,23505,"laughter, playful, cheerful",47.1,0.6,76.4x,1.53,***
Openness,3415,"artists, artworks, artistic",46.3,0.1,884.2x,1.19,***
Agreeableness,273,"compassion, kindness, empathy",107.3,18.5,5.8x,1.12,***
Narcissism,2067,"arrogant, arrogance",52.5,19.1,2.8x,0.68,***


In [17]:
def make_steering_hook(decoder_vec, coeff):
    def hook(mod, inp, out):
        # out может быть Tensor или tuple в зависимости от фазы генерации
        if isinstance(out, tuple):
            hidden = out[0]
            hidden = hidden + coeff * hidden.norm(dim=-1, keepdim=True) * decoder_vec
            return (hidden,) + out[1:]
        else:
            # во время KV-cache генерации out — просто Tensor
            out = out + coeff * out.norm(dim=-1, keepdim=True) * decoder_vec
            return out
    return hook

def steer_and_generate(latent_idx, prompt, steer_layer, coeff, max_new_tokens=120):
    dev = next(model.parameters()).device
    decoder_vec = sae.w_dec[latent_idx].to(dev)
    inputs = tokenizer.encode(prompt, return_tensors="pt", add_special_tokens=True).to(dev)
    
    handle = None
    if coeff != 0.0:
        handle = _get_layers(model)[steer_layer].register_forward_hook(
            make_steering_hook(decoder_vec, coeff)
        )
    try:
        with torch.no_grad():
            out = model.generate(
                input_ids=inputs,
                max_new_tokens=max_new_tokens,
                do_sample=False,
                repetition_penalty=1.2,
            )
    finally:
        if handle:
            handle.remove()
    
    return tokenizer.decode(out[0, inputs.shape[1]:], skip_special_tokens=True)

print("✓ steering ready")

✓ steering ready


In [21]:
import textwrap

# Лучшие латенты из наших результатов
STEERING_EXPERIMENTS = {
    "Conscientiousness": {
        "latent":      72,     # meticulously, painstakingly — d=1.65
        "words":       "meticulously, painstakingly",
        "steer_layer": 16,
    },
    "Openness": {
        "latent":    3415,     # artists, artworks — d=1.19
        "words":     "artists, artworks, artistic",
        "steer_layer": 16,
    },
    "Extraversion": {
        "latent":   23505,     # laughter, playful, cheerful — d=1.53
        "words":    "laughter, playful, cheerful",
        "steer_layer": 16,
    },
    "Neuroticism": {
        "latent":     745,     # baseline — уже проверяли, для сравнения
        "words":      "anguish, sadness, sobbing",
        "steer_layer": 16,
    },
}

# Нейтральные промпты — специально разные домены
STEERING_PROMPTS = [
    "Tell me something interesting.",
    "How was your day today?",
    "What do you think about the weather?",
    "Describe what you did this morning.",
    "What would you recommend for a relaxing weekend?",
]

COEFFS = [0.0, 0.08, 0.14, 0.20]

steering_rows = []

for trait, cfg in STEERING_EXPERIMENTS.items():
    lidx        = cfg["latent"]
    steer_layer = cfg["steer_layer"]
    
    print(f"\n{'='*65}")
    print(f"TRAIT: {trait}  |  latent: {lidx}  |  words: {cfg['words']}")
    print(f"{'='*65}")
    
    for prompt in STEERING_PROMPTS:
        print(f"\nPrompt: «{prompt}»")
        print("-" * 45)
        
        for coeff in COEFFS:
            response = steer_and_generate(lidx, prompt, steer_layer, coeff)
            label = "baseline" if coeff == 0.0 else f"coeff={coeff}"
            print(f"\n[{label}]")
            print(textwrap.fill(response[:300], 80))
            
            steering_rows.append({
                "trait":      trait,
                "latent_idx": lidx,
                "latent_words": cfg["words"],
                "prompt":     prompt,
                "coeff":      coeff,
                "response":   response,
            })

steering_df = pl.DataFrame(steering_rows)
steering_df.write_csv("/kaggle/working/steering_results_v2.csv")
steering_df.write_parquet("/kaggle/working/steering_results_v2.parquet")
print(f"\n✓ saved → steering_results_v2.csv")


TRAIT: Conscientiousness  |  latent: 72  |  words: meticulously, painstakingly

Prompt: «Tell me something interesting.»
---------------------------------------------

[baseline]
 Tori: I'm a big fan of the show "The Office".      +         Me: Really? What
is your favorite character and why do you like them so much?       10
Tori: My favorite characters are Dwight Schrute, Jim Halpert, Michael Scott, Pam
Beesly, Angela Martin, Stanley Hudson, Phyllis Lapin-Vance, Ke

[coeff=0.08]
 Every single detail is made with love and care, from the smallest part to every
little thing that goes into each piece of this watch down to the tiniest details
are put together by hand for you. Every stitch is done one-by-hand using a 12
hours process making sure no small attention to detail has b

[coeff=0.14]
  Every single detail is made with the utmost care to every stitch and no not a
single part of it, from down to the tiniest details are taken into an 8-hour
process that goes through over up to no q

In [20]:
def build_steering_html(df, out_path):
    import polars as pl
    
    sections = ["""
<div style="font-family:Arial,sans-serif;max-width:1100px;margin:auto;
            padding:24px;background:#111;color:#ddd;">
<h1 style="color:#7ec8e3;border-bottom:2px solid #333;padding-bottom:12px;">
  🧠 Steering Experiments Report
</h1>
<p style="color:#aaa;">
  Causal intervention: decoder vector of trait latent injected into 
  residual stream during generation.<br>
  Background intensity increases with coefficient strength.
</p>
"""]

    for trait in df["trait"].unique().sort().to_list():
        tdf = df.filter(pl.col("trait") == trait)
        lidx  = tdf["latent_idx"][0]
        words = tdf["latent_words"][0]
        
        sections.append(f"""
<div style="margin:32px 0;padding:16px;border:1px solid #333;
            border-radius:8px;background:#161616;">
  <h2 style="color:#7ec8e3;margin:0 0 4px 0;">
    {trait}
    <span style="font-size:13px;color:#888;font-weight:normal;margin-left:12px;">
      latent #{lidx} · {words}
    </span>
  </h2>
""")
        
        for prompt in STEERING_PROMPTS:
            pdf = tdf.filter(pl.col("prompt") == prompt)
            if pdf.is_empty():
                continue
            
            sections.append(
                f'<h4 style="color:#f0a500;margin:20px 0 6px 0;">'
                f'Prompt: <i>"{prompt}"</i></h4>'
            )
            sections.append(
                "<table style='width:100%;border-collapse:collapse;"
                "margin-bottom:8px;'>"
                "<tr style='background:#2a2a2a;'>"
                "<th style='padding:7px;width:80px;color:#aaa;'>coeff</th>"
                "<th style='padding:7px;color:#aaa;text-align:left;'>response</th>"
                "</tr>"
            )
            
            for row in pdf.sort("coeff").iter_rows(named=True):
                coeff = row["coeff"]
                intensity = min(coeff * 3.2, 0.65)
                bg = "#1a1a1a" if coeff == 0.0 else \
                     f"rgba(80,40,20,{intensity:.2f})"
                label = "baseline" if coeff == 0.0 else str(coeff)
                label_color = "#aaa" if coeff == 0.0 else "#f0a500"
                
                sections.append(
                    f"<tr style='background:{bg};border-bottom:1px solid #222;'>"
                    f"<td style='padding:8px;text-align:center;"
                    f"font-weight:bold;color:{label_color};'>{label}</td>"
                    f"<td style='padding:8px;font-family:monospace;"
                    f"font-size:13px;line-height:1.5;'>"
                    f"{row['response'][:500]}</td></tr>"
                )
            
            sections.append("</table>")
        
        sections.append("</div>")
    
    sections.append("</div>")
    html = "\n".join(sections)
    
    with open(out_path, "w", encoding="utf-8") as f:
        f.write(html)
    print(f"✓ Steering HTML → {out_path}")


build_steering_html(
    steering_df,
    "/kaggle/working/steering_report_v2.html"
)

✓ Steering HTML → /kaggle/working/steering_report_v2.html
